## 2.Data Cleaning & Feature Engineering

## Fit.ly Customer Churn Analysis

prepares cleaned, customer-level datasets by validating joins,
handling missing values, and engineering engagement and support features
for downstream analysis.

In [30]:
# Libraries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [31]:
# Ingest Raw data:

account_info = pd.read_csv("../data/raw/da_fitly_account_info.csv")
customer_support = pd.read_csv("../data/raw/da_fitly_customer_support.csv")
user_activity = pd.read_csv("../data/raw/da_fitly_user_activity.csv")

In [32]:
# structural overview:

datasets = {
    "Account Info": account_info,
    "Customer Support": customer_support,
    "User Activity": user_activity
}

for name, df in datasets.items():
    print(f"\n{name}")
    print("-" * len(name))
    print("Shape:", df.shape)
    display(df.head())
    
    df.info()

# I just need to check some few things for merging specialy datatypes


Account Info
------------
Shape: (400, 6)


,customer_id,email,state,plan,plan_list_price,churn_status
0,C10000,user10000@example.com,New Jersey,Enterprise,105,Y
1,C10001,user10001@example.net,Louisiana,Basic,22,Y
2,C10002,user10002@example.net,Oklahoma,Basic,24,NaN
3,C10003,user10003@example.com,Michigan,Free,0,NaN
4,C10004,user10004@example.com,Texas,Enterprise,119,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customer_id      400 non-null    object
 1   email            400 non-null    object
 2   state            400 non-null    object
 3   plan             400 non-null    object
 4   plan_list_price  400 non-null    int64 
 5   churn_status     114 non-null    object
dtypes: int64(1), object(5)
memory usage: 18.9+ KB

Customer Support
----------------
Shape: (918, 7)


,ticket_time,user_id,channel,topic,resolution_time_hours,state,comments
0,2025-06-13 05:55:17.154573,10125,chat,technical,11.48,1,NaN
1,2025-08-06 13:21:54.539551,10109,chat,account,1.01,0,NaN
2,2025-08-22 12:39:35.718663,10149,chat,technical,10.09,0,Erase my data from your systems.
3,2025-06-07 02:49:46.986055,10268,phone,account,9.10,1,NaN
4,2025-07-25 00:24:38.945079,10041,phone,other,2.28,1,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ticket_time            918 non-null    object 
 1   user_id                918 non-null    int64  
 2   channel                918 non-null    object 
 3   topic                  918 non-null    object 
 4   resolution_time_hours  918 non-null    float64
 5   state                  918 non-null    int64  
 6   comments               46 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 50.3+ KB

User Activity
-------------
Shape: (445, 3)


,event_time,user_id,event_type
0,2025-09-08 15:05:39.422721,10118,watch_video
1,2025-09-08 08:15:05.264103,10220,watch_video
2,2025-11-14 06:28:35.207671,10009,share_workout
3,2025-08-20 16:53:38.682901,10227,read_article
4,2025-07-24 16:47:31.728422,10123,track_workout


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_time  445 non-null    object
 1   user_id     445 non-null    int64 
 2   event_type  445 non-null    object
dtypes: int64(1), object(2)
memory usage: 10.6+ KB


## Data Type Standardization

Only necessary type conversions are applied to enable aggregation and joins.


### Data Type Fixese:


In [33]:
# Date & Time:

customer_support["ticket_time"] = pd.to_datetime(customer_support["ticket_time"])
user_activity["event_time"] = pd.to_datetime(user_activity["event_time"])

# customer_support
print(customer_support["ticket_time"].info())
print(customer_support["ticket_time"].head())

<class 'pandas.core.series.Series'>
RangeIndex: 918 entries, 0 to 917
Series name: ticket_time
Non-Null Count  Dtype         
--------------  -----         
918 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.3 KB
None
0   2025-06-13 05:55:17.154573
1   2025-08-06 13:21:54.539551
2   2025-08-22 12:39:35.718663
3   2025-06-07 02:49:46.986055
4   2025-07-25 00:24:38.945079
Name: ticket_time, dtype: datetime64[ns]


In [34]:
# user_activity

print(user_activity["event_time"].info())
print(user_activity["event_time"].head())

<class 'pandas.core.series.Series'>
RangeIndex: 445 entries, 0 to 444
Series name: event_time
Non-Null Count  Dtype         
--------------  -----         
445 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.6 KB
None
0   2025-09-08 15:05:39.422721
1   2025-09-08 08:15:05.264103
2   2025-11-14 06:28:35.207671
3   2025-08-20 16:53:38.682901
4   2025-07-24 16:47:31.728422
Name: event_time, dtype: datetime64[ns]


## Support Feature Engineering:

Support interactions are aggregated to the customer level to quantify friction.


In [35]:
# Aggregate Customer Level

support_agg = (
    customer_support
    .groupby("user_id")
    .agg(
        total_tickets=("user_id", "count"),
        avg_resolution_time=("resolution_time_hours", "mean")
    )
    .reset_index()
)
support_agg.head()

,user_id,total_tickets,avg_resolution_time
0,10000,3,21.446667
1,10001,4,17.542500
2,10002,3,6.433333
3,10003,1,2.190000
4,10004,4,8.542500


## Engagement Feature Engineering:

User activity logs are aggregated to capture engagement volume and behavior mix.


In [36]:
# Aggregate Activity Data:

activity_counts = (
    user_activity
    .groupby(["user_id", "event_type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Total engagement events
activity_counts["total_events"] = (
    activity_counts
    .drop(columns=["user_id"])
    .sum(axis=1)
)

activity_counts.head()

event_type,user_id,read_article,share_workout,track_workout,watch_video,total_events
0,10000,2,0,0,1,3
1,10001,1,0,0,0,1
2,10002,1,0,0,0,1
3,10003,0,0,1,0,1
4,10004,0,0,1,3,4


In [37]:
# add count the events frequency

activity_counts = (
    user_activity
    .groupby("user_id")
    .size()
    .reset_index(name="event_count")
)

activity_counts.head()

,user_id,event_count
0,10000,3
1,10001,1
2,10002,1
3,10003,1
4,10004,4


## Dataset Assembly:

All engineered features are merged into a single customer-level table.


In [38]:
# normalized join key

account_info["user_id"] = (
    account_info["customer_id"]
    .str.replace("C", "", regex=False)
    .astype(int)
)

account_info.head()

,customer_id,email,state,plan,plan_list_price,churn_status,user_id
0,C10000,user10000@example.com,New Jersey,Enterprise,105,Y,10000
1,C10001,user10001@example.net,Louisiana,Basic,22,Y,10001
2,C10002,user10002@example.net,Oklahoma,Basic,24,NaN,10002
3,C10003,user10003@example.com,Michigan,Free,0,NaN,10003
4,C10004,user10004@example.com,Texas,Enterprise,119,NaN,10004


In [39]:
# customer_support

customer_support.head()

,ticket_time,user_id,channel,topic,resolution_time_hours,state,comments
0,2025-06-13 05:55:17.154573,10125,chat,technical,11.48,1,NaN
1,2025-08-06 13:21:54.539551,10109,chat,account,1.01,0,NaN
2,2025-08-22 12:39:35.718663,10149,chat,technical,10.09,0,Erase my data from your systems.
3,2025-06-07 02:49:46.986055,10268,phone,account,9.10,1,NaN
4,2025-07-25 00:24:38.945079,10041,phone,other,2.28,1,NaN


In [40]:
# user_activity:

user_activity.head()

,event_time,user_id,event_type
0,2025-09-08 15:05:39.422721,10118,watch_video
1,2025-09-08 08:15:05.264103,10220,watch_video
2,2025-11-14 06:28:35.207671,10009,share_workout
3,2025-08-20 16:53:38.682901,10227,read_article
4,2025-07-24 16:47:31.728422,10123,track_workout


In [62]:
# support_agg.head()

In [63]:
# account_info.head()

In [60]:
# Merge Everything

final_df = (
    account_info
    .merge(support_agg, on="user_id", how="left")
    .merge(activity_counts, on="user_id", how="left")
    # .merge(user_activity, on="user_id", how="left")
    # .merge(customer_support, on="user_id", how="left")
)

# Missing Value Handling:

final_df[["ticket_count", "avg_resolution_time", "event_count"]] = (
    final_df[["total_tickets", "avg_resolution_time", "event_count"]].fillna(0)
)

# fill missing values:

final_df["event_count"] = final_df["event_count"].fillna(0)

In [61]:
final_df.columns
print(final_df.info())
final_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          400 non-null    object 
 1   email                400 non-null    object 
 2   state                400 non-null    object 
 3   plan                 400 non-null    object 
 4   plan_list_price      400 non-null    int64  
 5   churn_status         114 non-null    object 
 6   user_id              400 non-null    int64  
 7   total_tickets        367 non-null    float64
 8   avg_resolution_time  400 non-null    float64
 9   event_count          400 non-null    float64
 10  ticket_count         400 non-null    float64
dtypes: float64(4), int64(2), object(5)
memory usage: 34.5+ KB
None


,customer_id,email,state,plan,plan_list_price,churn_status,user_id,total_tickets,avg_resolution_time,event_count,ticket_count
0,C10000,user10000@example.com,New Jersey,Enterprise,105,Y,10000,3.0,21.446667,3.0,3.0
1,C10001,user10001@example.net,Louisiana,Basic,22,Y,10001,4.0,17.542500,1.0,4.0
2,C10002,user10002@example.net,Oklahoma,Basic,24,NaN,10002,3.0,6.433333,1.0,3.0
3,C10003,user10003@example.com,Michigan,Free,0,NaN,10003,1.0,2.190000,1.0,1.0
4,C10004,user10004@example.com,Texas,Enterprise,119,NaN,10004,4.0,8.542500,4.0,4.0


## Save File:

In [58]:
# Save point:

final_df.to_csv("../data/processed/fitly_customer_features.csv", index=False)

## Data Cleaning & Feature Engineering SUmmary:

Data preparation focused on ensuring consistency across multiple source systems and transforming raw logs into analysis-ready features. Customer identifiers were standardized by deriving a normalized user_id to resolve format mismatches between datasets while preserving original identifiers for reporting.

Missing and non-informative fields were handled explicitly, with qualitative fields excluded from quantitative analysis where appropriate. Support and activity data were aggregated to the customer level to maintain a one-row-per-customer structure and prevent duplication during merges.

Engineered features capture key behavioral signals, including support ticket volume, average resolution time, and user activity counts. These transformations result in a clean, consolidated dataset suitable for customer-level analysis and pattern identification.